In [2]:
import pandas as pd

In [4]:
df = pd.read_pickle('../data/Truck_ride.pkl')

In [15]:
df

,vin,triggerType_triggerType,triggerType_context,createdDateTime,receivedDateTime,hrTotalVehicleDistance,totalEngineHours,grossCombinationVehicleWeight,snapshotData_gnssPosition_latitude,snapshotData_gnssPosition_longitude,...,snapshotData_wheelBasedSpeed,snapshotData_fuelType,snapshotData_fuelLevel1,snapshotData_catalystFuelLevel,snapshotData_driver1WorkingState,snapshotData_driver2WorkingState,snapshotData_ambientAirTemperature,uptimeData_engineCoolantTemperature,uptimeData_serviceBrakeAirPressureCircuit1,uptimeData_serviceBrakeAirPressureCircuit2
0,XLRTEF5300G450805,IGNITION_ON,RFMS,2024-03-27T05:30:02.334Z,2024-03-27T05:31:20.694Z,83648285,0,0,51.264967,4.283523,...,0.0,Diesel,0.0,99,REST,REST,20,21,2040,2040
1,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-03-27T05:30:19.634Z,2024-03-27T05:31:35.615Z,83648285,0,0,51.264968,4.283523,...,0.0,Diesel,99.2,0,REST,REST,0,0,0,0
2,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:31:00.010Z,2024-03-27T05:32:09.086Z,83648285,1703,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
3,XLRTEF5300G450805,ENGINE_ON,RFMS,2024-03-27T05:31:00.981Z,2024-03-27T05:32:10.642Z,83648285,0,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
4,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:33:44.881Z,2024-03-27T05:34:54.411Z,83648290,1703,655350,51.264961,4.283535,...,4.0,Diesel,99.2,99,WORK,REST,7,29,944,944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-06-04T07:29:51.710Z,2024-06-04T07:30:26.081Z,112132250,0,0,47.837051,7.395467,...,0.0,Diesel,25.2,0,WORK,DRIVER_AVAILABLE,0,0,0,0
3496,XLRTEF5300G450805,TIMER,RFMS,2024-06-04T07:29:55.060Z,2024-06-04T07:30:30.192Z,112132245,2161,38800,47.837001,7.395462,...,2.0,Diesel,25.2,80,DRIVE,DRIVER_AVAILABLE,13,84,1000,1000
3497,XLRTEF5300G450805,IGNITION_OFF,RFMS,2024-06-04T07:29:55.844Z,2024-06-04T07:30:31.691Z,0,0,0,47.837100,7.395470,...,0.0,Diesel,25.2,102,None,None,13,83,0,0
3498,XLRTEF5300G450805,ENGINE_OFF,RFMS,2024-06-04T07:29:58.481Z,2024-06-04T07:30:32.300Z,0,0,0,47.837050,7.395468,...,0.0,Diesel,25.2,102,None,None,13,83,0,0


In [ ]:
import base64
import requests 
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from sklearn.decomposition import PCA
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')


def get_truck_info(vin="XLRTEF5300G450805", start_date="2024-03-25", end_date="2024-06-03"):

    auth_URL = 'https://api.connect.daf.com/rfms/token'
    base_URL = 'https://api.connect.daf.com'

    username= "Hackathon_4@daftrucks.com"
    password = "Team4Hackathon!"
    creds = username +':' + password
    creds_bytes = creds.encode('ascii')
    base64_bytes = base64.b64encode(creds_bytes)
    base64_message = base64_bytes.decode('ascii')
    headers = {
    'Authorization': 'Basic ' + base64_message }

    r = requests.post(auth_URL, headers=headers, verify=False)
    jsondata = r.json()
    token = jsondata.get('access_token')

    datetype ='&datetype=received'
    triggerfilter = '&Triggerfilter=DRIVER_1_WORKING_STATE_CHANGED'

    header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/vnd.fmsstandard.com.vehiclestatuses.v3.0+json; UTF-8'}

    data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    while current_date <= end_date:
        starttime = current_date.strftime("%Y-%m-%d") + "T23:59:59.999Z"
        stoptime = (current_date + timedelta(days=1)).strftime("%Y-%m-%d") + "T23:59:59.999Z"

        myurl = base_URL+'/rfms/vehiclestatuses?vin=' + vin +'&starttime=' + starttime

        req = requests.get(myurl, headers=header, verify=False)
        # print(jsondata)
        jsondata=req.json()
        print(jsondata)



        for i in jsondata['vehicleStatusResponse']['vehicleStatuses']:
            dat=[]
            dat.append(i['vin'])
            dat.append(i['triggerType']['triggerType'])
            dat.append(i['triggerType']['context'])
            dat.append(i['createdDateTime'])
            dat.append(i['receivedDateTime'])
            try:
                dat.append(i['hrTotalVehicleDistance'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['totalEngineHours'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['grossCombinationVehicleWeight'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['gnssPosition']['latitude'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['longitude'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['heading'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['speed'])
            except Exception:
                dat.append(-9999)

            try:
                dat.append(i['snapshotData']['gnssPosition']['positionDateTime'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['wheelBasedSpeed'])
            except Exception:
                dat.append(0)

            dat.append(i['snapshotData']['fuelType'])

            try:
                dat.append(i['snapshotData']['fuelLevel1'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['catalystFuelLevel'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['snapshotData']['driver1WorkingState'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['driver2WorkingState'])
            except Exception:
                dat.append(None)

            try:
                dat.append(i['snapshotData']['ambientAirTemperature'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['uptimeData']['engineCoolantTemperature'])
            except Exception:
                dat.append(0)
            try:
                dat.append(i['uptimeData']['serviceBrakeAirPressureCircuit1'])
            except Exception:
                dat.append(0)

            try:
                dat.append(i['uptimeData']['serviceBrakeAirPressureCircuit2'])
            except Exception:
                dat.append(0)

            data.append(dat)

        current_date += timedelta(days=1)

    df = pd.DataFrame(data, columns=['vin', 'triggerType_triggerType',
                                        'triggerType_context','createdDateTime',"receivedDateTime",
                                        "hrTotalVehicleDistance","totalEngineHours","grossCombinationVehicleWeight",
                                        'snapshotData_gnssPosition_latitude','snapshotData_gnssPosition_longitude',
                                        "snapshotData_gnssPosition_heading","snapshotData_gnssPosition_speed",
                                        "snapshotData_gnssPosition_positionDateTime",
                                        'snapshotData_wheelBasedSpeed','snapshotData_fuelType',
                                        'snapshotData_fuelLevel1','snapshotData_catalystFuelLevel',
                                        'snapshotData_driver1WorkingState','snapshotData_driver2WorkingState',
                                        'snapshotData_ambientAirTemperature','uptimeData_engineCoolantTemperature',
                                        'uptimeData_serviceBrakeAirPressureCircuit1','uptimeData_serviceBrakeAirPressureCircuit2'])

    df.drop_duplicates(inplace=True)
    
    return df


def get_dates_online():
    
    # Get today's date
    today = datetime.today().date()

    # Calculate the date 70 days before today
    days_before = today - timedelta(days=70)

    # Format the dates as YYYY-MM-DD
    today_str = today.strftime('%Y-%m-%d')
    days_before_str = days_before.strftime('%Y-%m-%d')

    # Create the tuple
    return today_str, days_before_str

# get the dates
end_date, start_date = get_dates_online()

df_truck = get_truck_info(start_date=start_date, end_date=end_date)

In [9]:
df_truck

,vin,triggerType_triggerType,triggerType_context,createdDateTime,receivedDateTime,hrTotalVehicleDistance,totalEngineHours,grossCombinationVehicleWeight,snapshotData_gnssPosition_latitude,snapshotData_gnssPosition_longitude,...,snapshotData_wheelBasedSpeed,snapshotData_fuelType,snapshotData_fuelLevel1,snapshotData_catalystFuelLevel,snapshotData_driver1WorkingState,snapshotData_driver2WorkingState,snapshotData_ambientAirTemperature,uptimeData_engineCoolantTemperature,uptimeData_serviceBrakeAirPressureCircuit1,uptimeData_serviceBrakeAirPressureCircuit2
0,XLRTEF5300G450805,IGNITION_ON,RFMS,2024-03-27T05:30:02.334Z,2024-03-27T05:31:20.694Z,83648285,0,0,51.264967,4.283523,...,0.0,Diesel,0.0,99,REST,REST,20,21,2040,2040
1,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-03-27T05:30:19.634Z,2024-03-27T05:31:35.615Z,83648285,0,0,51.264968,4.283523,...,0.0,Diesel,99.2,0,REST,REST,0,0,0,0
2,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:31:00.010Z,2024-03-27T05:32:09.086Z,83648285,1703,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
3,XLRTEF5300G450805,ENGINE_ON,RFMS,2024-03-27T05:31:00.981Z,2024-03-27T05:32:10.642Z,83648285,0,655350,51.264968,4.283523,...,0.0,Diesel,99.2,99,WORK,REST,7,21,528,528
4,XLRTEF5300G450805,TIMER,RFMS,2024-03-27T05:33:44.881Z,2024-03-27T05:34:54.411Z,83648290,1703,655350,51.264961,4.283535,...,4.0,Diesel,99.2,99,WORK,REST,7,29,944,944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,XLRTEF5300G450805,DRIVER_1_WORKING_STATE_CHANGED,RFMS,2024-06-04T07:29:51.710Z,2024-06-04T07:30:26.081Z,112132250,0,0,47.837051,7.395467,...,0.0,Diesel,25.2,0,WORK,DRIVER_AVAILABLE,0,0,0,0
3496,XLRTEF5300G450805,TIMER,RFMS,2024-06-04T07:29:55.060Z,2024-06-04T07:30:30.192Z,112132245,2161,38800,47.837001,7.395462,...,2.0,Diesel,25.2,80,DRIVE,DRIVER_AVAILABLE,13,84,1000,1000
3497,XLRTEF5300G450805,IGNITION_OFF,RFMS,2024-06-04T07:29:55.844Z,2024-06-04T07:30:31.691Z,0,0,0,47.837100,7.395470,...,0.0,Diesel,25.2,102,None,None,13,83,0,0
3498,XLRTEF5300G450805,ENGINE_OFF,RFMS,2024-06-04T07:29:58.481Z,2024-06-04T07:30:32.300Z,0,0,0,47.837050,7.395468,...,0.0,Diesel,25.2,102,None,None,13,83,0,0
